# 👁️ Object Tracking with OpenCV + MobileNetSSD 


📦 Install OpenCV library

> 💡 Note: Run `pip install opencv-python` in the terminal, not in a code cell.


In [ ]:
!pip install opencv-python

 🔍 Model Used:
MobileNet-SSD (Caffe model)  
Source: https://github.com/chuanqi305/MobileNet-SSD

Person Tracking

In [2]:
import cv2

# === Load the MobileNet SSD model for person detection ===
net = cv2.dnn.readNetFromCaffe(
    r"C:\Users\iphon\Desktop\Training projects\object-tracking-opencv\deploy.prototxt.txt",
    r"C:\Users\iphon\Desktop\Training projects\object-tracking-opencv\mobilenet_iter_73000.caffemodel"
)

# === Load the video file or camera ===
# cap = cv2.VideoCapture(0) # Uncomment for webcam
cap = cv2.VideoCapture(r"C:\Users\iphon\Desktop\Training projects\object-tracking-opencv\video.MP4")

# === Check if video opened successfully ===
if not cap.isOpened():
    print("❌ Error: Could not open video file.")
    exit()

# === Get video properties for saving ===
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# === Create VideoWriter object to save output video ===
out = cv2.VideoWriter(
    'output.avi',
    cv2.VideoWriter_fourcc(*'XVID'),
    fps,
    (frame_width, frame_height)
)

# === Create a CSRT tracker ===
tracker = cv2.TrackerCSRT_create()
init_tracker = False

# === Start reading video frames ===
while True:
    ret, frame = cap.read()
    if not ret:
        break

    if not init_tracker:
        # === Perform object detection on the first frame ===
        blob = cv2.dnn.blobFromImage(frame, 0.007843, (300, 300), 127.5)
        net.setInput(blob)
        detections = net.forward()

        # === Loop through all detections ===
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            class_id = int(detections[0, 0, i, 1])

            # === Check if the detected object is a person and confidence > 0.5 ===
            if confidence > 0.5 and class_id == 15:
                box = detections[0, 0, i, 3:7] * \
                      [frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]]
                (x, y, x1, y1) = box.astype("int")
                bbox = (x, y, x1 - x, y1 - y)

                # === Initialize the tracker with the bounding box ===
                tracker.init(frame, bbox)
                init_tracker = True
                break
    else:
        # === Update tracker on new frame ===
        success, bbox = tracker.update(frame)
        if success:
            (x, y, w, h) = [int(i) for i in bbox]
            
            # === Draw the bounding box and label ===
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, "Person", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # === Save the frame to output video ===
    out.write(frame)

    # === Display the frame ===
    cv2.imshow("People Tracking", frame)

    # === Exit when ESC key is pressed ===
    if cv2.waitKey(1) & 0xFF == 27:
        break

# === Release resources ===
cap.release()
out.release()
cv2.destroyAllWindows()


## 📌 Final Notes

- Press `ESC` to stop the tracking.
- You can switch between webcam or video file by commenting/uncommenting the related line.
- Tested on Python 3.10 + OpenCV 4.x
